In [ ]:
from sklearn.model_selection import train_test_split
%load_ext autoreload
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import prepare_dataframe, sentences_by_lang
df = prepare_dataframe()

### Przygotowanie danych

In [ ]:
from utils import languages, create_feature_dictionary, trigrams_by_lang

X_train: [str] = []
X_test: [str] = []
y_train: [str]= []
y_test: [str]= []

words = set()

for lang in languages:
    sentences = df[df["lang"] == lang]["sentence"]
    Xt, xtest = train_test_split(sentences, test_size = 0.2)
    yt =  len(Xt) * [lang]
    ytest =  len(xtest) * [lang]
    X_train.extend(Xt)
    y_train.extend(yt)
    X_test.extend(xtest)
    y_test.extend(ytest)

    vectorizer = CountVectorizer(max_features=trigrams_by_lang, min_df=5)
    X = vectorizer.fit_transform(Xt)
    words.update(vectorizer.vocabulary_)

vocabulary = create_feature_dictionary(words)

In [ ]:
vectorizer = CountVectorizer(vocabulary=vocabulary)
X = vectorizer.fit_transform(X_train)
features = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
from utils import normalize

normalize(features)

In [ ]:
from utils import FFN_Hyperparams, build_model
params = FFN_Hyperparams(len(features.columns), len(languages), [500,500,250], 'relu')
model = build_model(params)

In [ ]:
from utils import create_encoder, encode

encoder = create_encoder()
y = encode(y_train, encoder)

In [ ]:
from utils import DataGenerator

train_gen = DataGenerator(features, y, 32)
history = model.fit(train_gen, epochs=30, batch_size=6)

3000/3000 [==============================] - 13s 4ms/step - loss: 0.0854 - accuracy: 0.9618
Epoch 26/30
1183/3000 [==========>...................] - ETA: 8s - loss: 0.0823 - accuracy: 0.9633

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Trafność modelu na podstawie słów')
plt.ylabel('Trafność')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.title('Funkcja kosztu')
plt.ylabel('Koszt')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from utils import test_model

vectorizer = CountVectorizer(vocabulary=vocabulary)
X = vectorizer.fit_transform(X_test)
features = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())

encoder = create_encoder()
y = encode(y_test, encoder)

In [ ]:
accuracy = test_model(model, encoder, features, y)

In [ ]:
accuracy

In [ ]:
import pickle

print("Accuracy: ", accuracy)
if accuracy > 0.9:
    model.save("words_recognition")
    with open('words_labels.pkl', 'wb') as f:
        pickle.dump(encoder.classes_, f)
    with open('words_vectorizer.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)